In [ ]:
# %% === < Mounting File in Google Drive > ===
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !pip install tensorflow==2.4.0

In [ ]:
import tensorflow as tf
print(tf.__version__)

from keras.preprocessing import image_dataset_from_directory
batch_size = 101
# image_size = (299, 299) # InceptionV3
image_size = (224, 224) # ResNet50, MobileNetV3
DataFolder = '/content/drive/MyDrive/Info/data/representative_images'
RepresentativeDataset = image_dataset_from_directory(DataFolder, 
    labels='inferred',
    label_mode='int',
    color_mode='rgb',
    image_size=image_size,
    batch_size=batch_size,
    seed=0,
    shuffle=True
    )

2.4.1
Found 303 files belonging to 101 classes.


In [ ]:
# n=0
# for images, _ in RepresentativeDataset.take(100):
#   n+=1
#   print(n)

In [ ]:
def representative_dataset():
  for images, _ in RepresentativeDataset.take(100):
    yield [images.numpy().astype("float32")]

In [ ]:
# === Convert the model
saved_model_dir = '/content/drive/MyDrive/Info/result/model_weights/food101_InceptionV3_20210330_1015.h5'
model = tf.keras.models.load_model(saved_model_dir)
converter = tf.lite.TFLiteConverter.from_keras_model(model) # path to the SavedModel directory
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8  # or tf.int8
converter.inference_output_type = tf.uint8  # or tf.int8
tflite_quant_model = converter.convert()

# === Save the model.
tflite_dir = '/content/drive/MyDrive/Info/tflite_with_metadata/food101_InceptionV3_20210330_1015_uint8_v2.tflite'
with open(tflite_dir, 'wb') as f:
  f.write(tflite_quant_model)

###
print('Finish!!!')

INFO:tensorflow:Assets written to: /tmp/tmpwzqxlf8l/assets
Finish!!!
